<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Mask_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ftaubner/semantic_features_detection.git

Cloning into 'semantic_features_detection'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 1110 (delta 52), reused 32 (delta 13), pack-reused 1017
Receiving objects: 100% (1110/1110), 113.90 MiB | 46.65 MiB/s, done.
Resolving deltas: 100% (658/658), done.


In [0]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199865&authkey=ALxjJOloi9Wjk80" -O mapillary_vistas_sample.zip

--2020-04-01 10:49:08--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199865&authkey=ALxjJOloi9Wjk80
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yijzyw.am.files.1drv.com/y4m-tCRsQ87By4y4UVK6F-lSBLElW53N2LOPFRiFuLEuYRLtI25YJyq74Te4TWK-3LsXiModWkhiquJYDNJT02NCvmWMtriKXgDvVm_lQJHop57CwzKB9q6f8wvWn6arI8qXGpRONkZsUIwsZ0lYgfD1jc_nLhAl0X6hUE_4Sn_rP7TohzFnfe764gQznxZNR2F0VX9fgpwBfwblSLGxeVnQw/mapillary_vistas_sample.zip?download&psid=1 [following]
--2020-04-01 10:49:08--  https://yijzyw.am.files.1drv.com/y4m-tCRsQ87By4y4UVK6F-lSBLElW53N2LOPFRiFuLEuYRLtI25YJyq74Te4TWK-3LsXiModWkhiquJYDNJT02NCvmWMtriKXgDvVm_lQJHop57CwzKB9q6f8wvWn6arI8qXGpRONkZsUIwsZ0lYgfD1jc_nLhAl0X6hUE_4Sn_rP7TohzFnfe764gQznxZNR2F0VX9fgpwBfwblSLGxeVnQw/mapillary_vistas_sample.zip?download&psid=1
Resolving yijzyw.am.files.1dr

In [0]:
!unzip -qq /content/mapillary_vistas_sample.zip -d /content

In [0]:
!pip install imgaug
!pip install Cython
!pip install pycocotools

In [0]:
import os
os.chdir('/content/semantic_features_detection/Notebook')

In [0]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import imageio
%tensorflow_version 1.x
import tensorflow as tf


# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Ignore Warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
print(ROOT_DIR)
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/mapillary/"))  # To find local version
import mapillary

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
print(MODEL_DIR)

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
print('finished')

In [0]:

TRAIN_DIR = '/content/mapillary_vistas'
# Training dataset
dataset_train = mapillary.MapillaryDataset()
dataset_train.load_vistas(dataset_dir=TRAIN_DIR, subset='training')
dataset_train.prepare()

# Validation dataset
VAL_DIR = '/content/mapillary_vistas'

dataset_val = mapillary.MapillaryDataset()
dataset_val.load_vistas(dataset_dir=VAL_DIR, subset='validation')
dataset_val.prepare()

In [0]:
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    print(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [0]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
#model.load_weights(model_path, by_name=True)

# Load pretrained weights
model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])

In [0]:
class TrainConfig(mapillary.mapvistas):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 10

config = TrainConfig()
config.display()
batch_size = config.BATCH_SIZE

In [0]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=MODEL_DIR, histogram_freq=1)

In [0]:
# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='heads')

In [0]:
# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=120,
            layers='4+')



In [0]:
# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=160,
            layers='all')